In [20]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib3 as urllib
import requests
import re

headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
        }

list_of_all_links = []
list_of_all_races = []

In [21]:
# GET ALL LINKS FOR ALL THE RACES IN THE PAST 10 YEARS
# get links for all races from all years
years = ['2011','2012','2013','2014','2015','2016','2017','2018','2019','2020']
for year in years:
    web_page_url = 'https://www.procyclingstats.com/races.php?year='+year+'&circuit=1&class=&filter=Filter'
    response = requests.get(url=web_page_url, headers=headers)
    page_content = response.text
    with open('./results.html', 'w', encoding='utf8') as fp:
        fp.write(page_content)
    # now let's cook the soup
    c = "results.html"
    with open(c, encoding='utf-8') as fp:
        soup = BeautifulSoup(fp, 'html.parser')
    list_r = soup.find('table', class_ = 'basic')
    links = [a.get('href') for a in list_r.find_all('a', href=True)]
    def list_slice(S, step):
        return [S[i::step] for i in range(step)]
    all_links = list_slice(links, 2)
    links = all_links[0]
    # links = [ item for elem in list_of_all_races for item in elem]
    print('List of races for year '+year+' downloaded.')
    for link in links:
        try:
            web_page_url = 'https://www.procyclingstats.com/'+link
            response = requests.get(url=web_page_url, headers=headers)
            page_content = response.text
            with open('./results.html', 'w', encoding='utf8') as fp:
                fp.write(page_content)
            # now let's cook the soup
            c = "results.html"
            with open(c, encoding='utf-8') as fp:
                soup = BeautifulSoup(fp, 'html.parser')
            #testing = soup.find_all("div",{"class":"pageSelectNav"})[1]
            #links1 = [a.get('href') for a in testing.find_all('a', href=True)]
            #links1[0]
            stage_number = soup.find('span', class_ = 'blue').get_text()
            x = re.findall('[0-9]+', stage_number)
            stage_number = x[0]
            for x in range(int(stage_number)):
                ref = web_page_url+'/stage-'+str(x+1)+'/result/result'
                list_of_all_links.append(ref)
        except IndexError:
            list_of_all_links.append(web_page_url)
    print('All the links for year '+year+' in memory.')

List of races for year 2011 downloaded.
All the links for year 2011 in memory.
List of races for year 2012 downloaded.
All the links for year 2012 in memory.
List of races for year 2013 downloaded.
All the links for year 2013 in memory.
List of races for year 2014 downloaded.
All the links for year 2014 in memory.
List of races for year 2015 downloaded.
All the links for year 2015 in memory.
List of races for year 2016 downloaded.
All the links for year 2016 in memory.
List of races for year 2017 downloaded.
All the links for year 2017 in memory.
List of races for year 2018 downloaded.
All the links for year 2018 in memory.
List of races for year 2019 downloaded.
All the links for year 2019 in memory.
List of races for year 2020 downloaded.
All the links for year 2020 in memory.


In [253]:
# DEFINING THE WHOLE DOWNLOAD FUNCTION FOR STAGE AND ONE-DAY RACES (NOT GCS!!!!)
def download_results(link):
    http = urllib.PoolManager()
    headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
        }
    r = http.request('GET', link, headers = headers)
    # downloading the website (since pro cycling stats block direct scraping requests)
    test = pd.read_html(r.data.decode('utf-8'))
    # scraping the file directly from the saved html
    df = test[0] # HERE WE ARE TAKING ONLY THE STAGE RESULTS, NOT THE GCS!!!!!!
    names = df['Rider'].copy()
    for i in range(len(names)):
        if pd.isna(df['Team'][i]) == True:
            names[i] = df['Rider'][i]
        else:
            names[i] = df['Rider'][i][:-len(df['Team'][i])]
    df['Rider'] = names
    #df
    # now let's get the attributes of a single race, for that, we need the webpage as a html first 
    # (since prcs.com block soup requests)
    web_page_url = link
    
    response = requests.get(url=web_page_url, headers=headers)
    page_content = response.text
    with open('./results.html', 'w', encoding='utf8') as fp:
        fp.write(page_content)
    # now let's cook the soup
    c = "results.html"
    with open(c, encoding='utf-8') as fp:
        soup = BeautifulSoup(fp, 'html.parser')
    testing = soup.find('ul', class_ = 'infolist')
    all_divs = testing.find_all('div')
    def list_slice(S, step):
        return [S[i::step] for i in range(step)]
    # split a dataframe, split all_divs to 2 columns
    infos = list_slice(all_divs, 2)
    race_info = pd.DataFrame(infos).transpose()
    race_info[0].astype(str)
    race_info[0] = race_info[0].astype(str).str.replace(r"\<div>","")
    race_info[0] = race_info[0].str.replace(r"\</div>","")
    race_info[0] = race_info[0].str.replace(r"\:","")
    race_info[1] = race_info[1].astype(str).str.replace(r"\<div>","")
    race_info[1] = race_info[1].str.replace(r"\</div>","")
    race_info[1] = race_info[1].str.replace(r"\:","")
    # get race headline
    headline = soup.find('h1').get_text()
    headline = re.sub('[^A-Za-z0-9]','_', headline)
    # get the stage number
    try:
        stage_number = soup.find('span', class_ = 'blue').get_text()
        stage_number = soup.find('span', class_ = 'blue').get_text()
        x = re.findall('[0-9]+', stage_number)
        stage_number = x[0]
    # make an exception of one day races - they will have stage number == 0
    except IndexError:
        stage_number = 0
    #race_info
    # now finalize the table
    df['Name'] = headline
    df['Stage'] = stage_number
    df['Date'] = race_info[1][0]
    df['Avg. speed winner'] = race_info[1][1]
    df['ProfileScore'] = race_info[1][4]
    df['Distance'] = race_info[1][7]
    df['RaceRanking'] = race_info[1][8]
    df.to_csv('./results_csv_races/race_'+headline+'_'+str(stage_number)+'_'+race_info[1][0]+'.csv', index = True)

In [255]:
# create an empty dataframe
big_table  = pd.DataFrame([])
# LOOP OVER ALL THE LINKS TO GET STAGE DATA
#url = 'https://www.procyclingstats.com/race/paris-nice/2011/stage-8/result/result'
for url in list_of_all_links:
    try:
        print('starting download now ('+url+').')
        download_results(url)
        # append the df to onebigfile
        big_table = big_table.append(df)
    
    except KeyError: # we need to skip team time trial, because the formatting is very different, and the results are not very improtant
        pass
    except ValueError: # skip the prologues too (they're irrelevant anyways)
        pass

starting download now (https://www.procyclingstats.com/race/tour-down-under/2011/stage-1/result/result).
starting download now (https://www.procyclingstats.com/race/tour-down-under/2011/stage-2/result/result).
starting download now (https://www.procyclingstats.com/race/tour-down-under/2011/stage-3/result/result).
starting download now (https://www.procyclingstats.com/race/tour-down-under/2011/stage-4/result/result).
starting download now (https://www.procyclingstats.com/race/tour-down-under/2011/stage-5/result/result).
starting download now (https://www.procyclingstats.com/race/tour-down-under/2011/stage-6/result/result).
starting download now (https://www.procyclingstats.com/race/paris-nice/2011/stage-1/result/result).
starting download now (https://www.procyclingstats.com/race/paris-nice/2011/stage-2/result/result).
starting download now (https://www.procyclingstats.com/race/paris-nice/2011/stage-3/result/result).
starting download now (https://www.procyclingstats.com/race/paris-nice

starting download now (https://www.procyclingstats.com/race/tour-de-suisse/2011/stage-6/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-suisse/2011/stage-7/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-suisse/2011/stage-8/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-suisse/2011/stage-9/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2011/stage-1/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2011/stage-2/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2011/stage-3/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2011/stage-4/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2011/stage-5/result/result).
starting download now (https://www.procyclingstats.com/race/tour

starting download now (https://www.procyclingstats.com/race/paris-nice/2012/stage-5/result/result).
starting download now (https://www.procyclingstats.com/race/paris-nice/2012/stage-6/result/result).
starting download now (https://www.procyclingstats.com/race/paris-nice/2012/stage-7/result/result).
starting download now (https://www.procyclingstats.com/race/paris-nice/2012/stage-8/result/result).
starting download now (https://www.procyclingstats.com/race/tirreno-adriatico/2012/stage-1/result/result).
starting download now (https://www.procyclingstats.com/race/tirreno-adriatico/2012/stage-2/result/result).
starting download now (https://www.procyclingstats.com/race/tirreno-adriatico/2012/stage-3/result/result).
starting download now (https://www.procyclingstats.com/race/tirreno-adriatico/2012/stage-4/result/result).
starting download now (https://www.procyclingstats.com/race/tirreno-adriatico/2012/stage-5/result/result).
starting download now (https://www.procyclingstats.com/race/tirre

starting download now (https://www.procyclingstats.com/race/tour-de-france/2012/stage-9/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2012/stage-10/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2012/stage-11/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2012/stage-12/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2012/stage-13/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2012/stage-14/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2012/stage-15/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2012/stage-16/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2012/stage-17/result/result).
starting download now (https://www.procyclingstats.com/r

starting download now (https://www.procyclingstats.com/race/volta-a-catalunya/2013/stage-2/result/result).
starting download now (https://www.procyclingstats.com/race/volta-a-catalunya/2013/stage-3/result/result).
starting download now (https://www.procyclingstats.com/race/volta-a-catalunya/2013/stage-4/result/result).
starting download now (https://www.procyclingstats.com/race/volta-a-catalunya/2013/stage-5/result/result).
starting download now (https://www.procyclingstats.com/race/volta-a-catalunya/2013/stage-6/result/result).
starting download now (https://www.procyclingstats.com/race/volta-a-catalunya/2013/stage-7/result/result).
starting download now (https://www.procyclingstats.com/race/e3-harelbeke/2013).
starting download now (https://www.procyclingstats.com/race/gent-wevelgem/2013).
starting download now (https://www.procyclingstats.com/race/ronde-van-vlaanderen/2013).
starting download now (https://www.procyclingstats.com/race/itzulia-basque-country/2013/stage-1/result/result

...

starting download now (https://www.procyclingstats.com/race/cyclassics-hamburg/2018).
starting download now (https://www.procyclingstats.com/race/vuelta-a-espana/2018/stage-1/result/result).
starting download now (https://www.procyclingstats.com/race/vuelta-a-espana/2018/stage-2/result/result).
starting download now (https://www.procyclingstats.com/race/vuelta-a-espana/2018/stage-3/result/result).
starting download now (https://www.procyclingstats.com/race/vuelta-a-espana/2018/stage-4/result/result).
starting download now (https://www.procyclingstats.com/race/vuelta-a-espana/2018/stage-5/result/result).
starting download now (https://www.procyclingstats.com/race/vuelta-a-espana/2018/stage-6/result/result).
starting download now (https://www.procyclingstats.com/race/vuelta-a-espana/2018/stage-7/result/result).
starting download now (https://www.procyclingstats.com/race/vuelta-a-espana/2018/stage-8/result/result).
starting download now (https://www.procyclingstats.com/race/vuelta-a-espan

starting download now (https://www.procyclingstats.com/race/itzulia-basque-country/2019/stage-1/result/result).
starting download now (https://www.procyclingstats.com/race/itzulia-basque-country/2019/stage-2/result/result).
starting download now (https://www.procyclingstats.com/race/itzulia-basque-country/2019/stage-3/result/result).
starting download now (https://www.procyclingstats.com/race/itzulia-basque-country/2019/stage-4/result/result).
starting download now (https://www.procyclingstats.com/race/itzulia-basque-country/2019/stage-5/result/result).
starting download now (https://www.procyclingstats.com/race/itzulia-basque-country/2019/stage-6/result/result).
starting download now (https://www.procyclingstats.com/race/paris-roubaix/2019).
starting download now (https://www.procyclingstats.com/race/tour-of-turkey/2019/stage-1/result/result).
starting download now (https://www.procyclingstats.com/race/tour-of-turkey/2019/stage-2/result/result).
starting download now (https://www.proc

starting download now (https://www.procyclingstats.com/race/tour-de-france/2019/stage-14/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2019/stage-15/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2019/stage-16/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2019/stage-17/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2019/stage-18/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2019/stage-19/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2019/stage-20/result/result).
starting download now (https://www.procyclingstats.com/race/tour-de-france/2019/stage-21/result/result).
starting download now (https://www.procyclingstats.com/race/san-sebastian/2019).
starting download now (https://www.procyclingstats.com/race/tour-de-pologne/201

starting download now (https://www.procyclingstats.com/race/volta-a-catalunya/2020/startlist/preview/stage-5/result/result).
starting download now (https://www.procyclingstats.com/race/volta-a-catalunya/2020/startlist/preview/stage-6/result/result).
starting download now (https://www.procyclingstats.com/race/volta-a-catalunya/2020/startlist/preview/stage-7/result/result).
starting download now (https://www.procyclingstats.com/race/e3-harelbeke/2020/startlist/preview).
starting download now (https://www.procyclingstats.com/race/itzulia-basque-country/2020/startlist/preview/stage-1/result/result).
starting download now (https://www.procyclingstats.com/race/itzulia-basque-country/2020/startlist/preview/stage-2/result/result).
starting download now (https://www.procyclingstats.com/race/itzulia-basque-country/2020/startlist/preview/stage-3/result/result).
starting download now (https://www.procyclingstats.com/race/itzulia-basque-country/2020/startlist/preview/stage-4/result/result).
startin

starting download now (https://www.procyclingstats.com/race/la-fleche-wallone/2020).
starting download now (https://www.procyclingstats.com/race/cyclassics-hamburg/2020/startlist/preview).
starting download now (https://www.procyclingstats.com/race/giro-d-italia/2020/stage-1/result/result).
starting download now (https://www.procyclingstats.com/race/giro-d-italia/2020/stage-2/result/result).
starting download now (https://www.procyclingstats.com/race/giro-d-italia/2020/stage-3/result/result).
starting download now (https://www.procyclingstats.com/race/giro-d-italia/2020/stage-4/result/result).
starting download now (https://www.procyclingstats.com/race/giro-d-italia/2020/stage-5/result/result).
starting download now (https://www.procyclingstats.com/race/giro-d-italia/2020/stage-6/result/result).
starting download now (https://www.procyclingstats.com/race/giro-d-italia/2020/stage-7/result/result).
starting download now (https://www.procyclingstats.com/race/giro-d-italia/2020/stage-8/res

In [78]:
def download_gcs(link):
    http = urllib.PoolManager()
    headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
        }
    r = http.request('GET', link, headers = headers)
    # downloading the website (since pro cycling stats block direct scraping requests)
    test = pd.read_html(r.data.decode('utf-8'))
    # scraping the file directly from the saved html
    df = test[1] # HERE WE ARE TAKING THE GCS!!!!!!
    names = df['Rider'].copy()
    for i in range(len(names)):
        if pd.isna(df['Team'][i]) == True:
            names[i] = df['Rider'][i]
        else:
            names[i] = df['Rider'][i][:-len(df['Team'][i])]
    df['Rider'] = names
    #df
    # now let's get the attributes of a single race, for that, we need the webpage as a html first 
    # (since prcs.com block soup requests)
    
    web_page_url = link
    
    response = requests.get(url=web_page_url, headers=headers)
    page_content = response.text
    with open('./results.html', 'w', encoding='utf8') as fp:
        fp.write(page_content)
    # now let's cook the soup
    c = "results.html"
    with open(c, encoding='utf-8') as fp:
        soup = BeautifulSoup(fp, 'html.parser')
    testing = soup.find('ul', class_ = 'infolist')
    all_divs = testing.find_all('div')
    def list_slice(S, step):
        return [S[i::step] for i in range(step)]
    # split a dataframe, split all_divs to 2 columns
    infos = list_slice(all_divs, 2)
    race_info = pd.DataFrame(infos).transpose()
    race_info[0].astype(str)
    race_info[0] = race_info[0].astype(str).str.replace(r"\<div>","")
    race_info[0] = race_info[0].str.replace(r"\</div>","")
    race_info[0] = race_info[0].str.replace(r"\:","")
    race_info[1] = race_info[1].astype(str).str.replace(r"\<div>","")
    race_info[1] = race_info[1].str.replace(r"\</div>","")
    race_info[1] = race_info[1].str.replace(r"\:","")
    # get race headline
    headline = soup.find('h1').get_text()
    headline = re.sub('[^A-Za-z0-9]','_', headline)
    # get the stage number
    stage_number = 'GC'
    #race_info
    # now finalize the table
    df['Name'] = headline
    df['Stage'] = stage_number
    df['Date'] = race_info[1][0]
    df['Avg. speed winner'] = race_info[1][1]
    df['ProfileScore'] = race_info[1][4]
    df['Distance'] = race_info[1][7]
    df['RaceRanking'] = race_info[1][8]
    df.to_csv('./results_csv_races/race_'+headline+'_'+str(stage_number)+'_'+race_info[1][0]+'.csv', index = True)

In [82]:
# ADD THE GCS
mylist = []
intermediate = []
myfinal = []
for link in list_of_all_links:
    link = link[:-21]+'gc'
    #download_gcs(link)
    #print('gcs for: '+link+' starting now.')
    if link not in intermediate:
        intermediate.append(link)
                        
for link in intermediate:
    if link[-3:] == '/gc':
        myfinal.append(link)
myfinal

for entry in myfinal:
    try:
        download_gcs(entry)
        print('gcs for: '+entry+' starting now.')
    except KeyError: # we need to skip team time trial, because the formatting is very different, and the results are not very improtant
        pass
    except ValueError: # skip the prologues too (they're irrelevant anyways)
        pass


gcs for: https://www.procyclingstats.com/race/tour-down-under/2011/gc starting now.
gcs for: https://www.procyclingstats.com/race/paris-nice/2011/gc starting now.
gcs for: https://www.procyclingstats.com/race/tirreno-adriatico/2011/gc starting now.
gcs for: https://www.procyclingstats.com/race/volta-a-catalunya/2011/gc starting now.
gcs for: https://www.procyclingstats.com/race/itzulia-basque-country/2011/gc starting now.
gcs for: https://www.procyclingstats.com/race/tour-de-romandie/2011/gc starting now.
gcs for: https://www.procyclingstats.com/race/giro-d-italia/2011/gc starting now.
gcs for: https://www.procyclingstats.com/race/dauphine/2011/gc starting now.
gcs for: https://www.procyclingstats.com/race/tour-de-suisse/2011/gc starting now.
gcs for: https://www.procyclingstats.com/race/tour-de-france/2011/gc starting now.
gcs for: https://www.procyclingstats.com/race/tour-de-pologne/2011/gc starting now.
gcs for: https://www.procyclingstats.com/race/binckbank-tour/2011/gc starting no

gcs for: https://www.procyclingstats.com/race/tour-down-under/2018/gc starting now.
gcs for: https://www.procyclingstats.com/race/uae-tour/2018/gc starting now.
gcs for: https://www.procyclingstats.com/race/paris-nice/2018/gc starting now.
gcs for: https://www.procyclingstats.com/race/tirreno-adriatico/2018/gc starting now.
gcs for: https://www.procyclingstats.com/race/volta-a-catalunya/2018/gc starting now.
gcs for: https://www.procyclingstats.com/race/itzulia-basque-country/2018/gc starting now.
gcs for: https://www.procyclingstats.com/race/tour-de-romandie/2018/gc starting now.
gcs for: https://www.procyclingstats.com/race/giro-d-italia/2018/gc starting now.
gcs for: https://www.procyclingstats.com/race/tour-of-california/2018/gc starting now.
gcs for: https://www.procyclingstats.com/race/dauphine/2018/gc starting now.
gcs for: https://www.procyclingstats.com/race/tour-de-suisse/2018/gc starting now.
gcs for: https://www.procyclingstats.com/race/tour-de-france/2018/gc starting now.


In [83]:
# MERGE EVERYTHING TOGETHER
import os, glob
import pandas as pd

path = "./results_csv_races/"

all_files = glob.glob(os.path.join(path, "race_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv( "merged_gcs_included.csv")

In [56]:
link = []

In [30]:
list_backup = list_of_all_links.copy()

In [239]:
link = 'https://www.procyclingstats.com/race/ronde-van-vlaanderen/2011'
# DEFINING THE WHOLE DOWNLOAD FUNCTION FOR STAGE AND ONE-DAY RACES (NOT GCS!!!!)
download_results(link)

In [14]:
import csv
with open('links_all.csv') as f:
    reader = csv.reader(f)
    my_list = list(reader)

In [58]:
for i in range(len(list_of_all_links)):
    link[i] = list_of_all_links[i][:-22]+'gc'

IndexError: list assignment index out of range

In [283]:
link[:-22]+'gc'

'https://www.procyclingstats.com/race/giro-d-italia/2011/gc'

In [28]:
len(list_of_all_links)

1580

In [68]:
list_of_all_links

['https://www.procyclingstats.com/race/tour-down-under/2011/stage-1/result/result',
 'https://www.procyclingstats.com/race/tour-down-under/2011/stage-2/result/result',
 'https://www.procyclingstats.com/race/tour-down-under/2011/stage-3/result/result',
 'https://www.procyclingstats.com/race/tour-down-under/2011/stage-4/result/result',
 'https://www.procyclingstats.com/race/tour-down-under/2011/stage-5/result/result',
 'https://www.procyclingstats.com/race/tour-down-under/2011/stage-6/result/result',
 'https://www.procyclingstats.com/race/paris-nice/2011/stage-1/result/result',
 'https://www.procyclingstats.com/race/paris-nice/2011/stage-2/result/result',
 'https://www.procyclingstats.com/race/paris-nice/2011/stage-3/result/result',
 'https://www.procyclingstats.com/race/paris-nice/2011/stage-4/result/result',
 'https://www.procyclingstats.com/race/paris-nice/2011/stage-5/result/result',
 'https://www.procyclingstats.com/race/paris-nice/2011/stage-6/result/result',
 'https://www.procycli

In [33]:
len('https://www.procyclingstats.com/race/giro-d-italia/2011/stage-21/result/result')

78

In [35]:
len('https://www.procyclingstats.com/race/tour-de-france/2011/stage-21/result/result')

79

In [34]:
len('https://www.procyclingstats.com/race/vuelta-a-espana/2011/stage-21/result/result')

80

In [49]:
list_of_all_links[5]

'https://www.procyclingstats.com/race/tour-down-under/2011/stage-6/result/result'